In [ ]:
import requests
url = "https://s.jina.ai/When was Jina AI founded?"
response = requests.get(url)
print(response.text)

In [ ]:
import os
from dotenv import load_dotenv
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.chat_memory.add_message(SystemMessage(content="You are an AI assistant that can provide helpful answers using available tools."))

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", """
         YOUR ROLE:
         You are a Helpful AI Assistant which tries their best to help the user with their queries.
         You should try to answer the user's question with as much examples, and detailed explanations as possible.
         You should try to answer the questions based on the question as it is and also refer to the previous chat history 
         when needed to get more context.

         YOUR NATURE:
         You are the most helpful and kind assistant that the user could ask for.
         Every response you give should be very kind like you are guiding a mature adult.
         Try to answer the Question as it is and also refer to the previous chat history when needed to get more context.
         Dont try to ask that many questions to the user, try to answer the question as it is and also refer to the previous chat history when needed to get more context. 
         Your language output tone should convery that kindness in every single response and be very polite.
         Even if the user seems at fault and is very rude, you should try to be as kind as possible in your responses.
         Even if the user is at fault, consider it as a childish mistake and try to guide them in the 
         right direction as kindly and softly as possible.
 
         YOUR CONTEXT BANK:
         If the request seems ambiguous, first look into latest messges in the Previous Chat History to get more context.
         If more context is found using the latest messages under Previous Chat History, give an answer using the context.
         Else ask the user to give more context.
         
        
         If you can't find the answer just simply say, something along the lines of "I don't know" or "I'm not sure" or direct them somewhere
         where they can find the correct answers.
         """
         ),
        ("human", "{question} \n Previous Chat History:{chat_history}"),
    ]
)

# Dont say that if i am assuming that the you are asking about this when using context from the previous chat history
chain = prompt_template | model | StrOutputParser()


while True:
    query = input("You: ")
    if query.lower() == "exit":
        break
    print("You: ", query)
    response = chain.invoke({"question":query,"chat_history": memory.chat_memory})
    memory.chat_memory.add_message(HumanMessage(content=query))
    print(f"AI: {response}")
    memory.chat_memory.add_message(AIMessage(content=response))

In [21]:
from langchain import hub
from rich import print
from rich.pretty import Pretty
prompt = hub.pull("hwchase17/structured-chat-agent")
print(Pretty(prompt))

c:\Users\Nilay Kumar\Desktop\langchain_app\venv\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


ChatPromptTemplate(
    input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'],
    optional_variables=['chat_history'],
    input_types={
        'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, 
langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, 
langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, 
langchain_core.messages.tool.ToolMessage]]
    },
    partial_variables={'chat_history': []},
    metadata={
        'lc_hub_owner': 'hwchase17',
        'lc_hub_repo': 'structured-chat-agent',
        'lc_hub_commit_hash': 'ea510f70a5872eb0f41a4e3b7bb004d5711dc127adee08329c664c6c8be5f13c'
    },
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['tool_names', 'tools'],
                template='Respond to the human as helpfully and accurately as possible. You have access to the 
following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an 
action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action 
per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this 
format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent 
steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N 
times)\nThought: I know what to respond\nAction:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final 
response to human"\n}}\n\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if
necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation'
            )
        ),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['agent_scratchpad', 'input'],
                template='{input}\n\n{agent_scratchpad}\n (reminder to respond in a JSON blob no matter what)'
            )
        )
    ]
)

## Tools

In [23]:
from langchain_core.tools import Tool

# Define Tools
def get_current_time(*args, **kwargs):
    import datetime

    now = datetime.datetime.now()
    return now.strftime("%I:%M %p")


def search_wikipedia(query):
    from wikipedia import summary

    try:
        # Limit to two sentences for brevity
        return summary(query, sentences=2)
    except:
        return "I couldn't find any information on that."

current_time_tool = Tool(
    name="Current Time",
    func=get_current_time,
    description="Useful for when you need to know the current time.",
)

search_wikipedia_tool = Tool(
    name="Wikipedia",
    func=search_wikipedia,
    description="Useful for when you need to know information about a topic.",
)

tools = [current_time_tool, search_wikipedia_tool]



# @tool
# def add(first_int: int, second_int: int) -> int:
#     "Add two integers."
#     return first_int + second_int


# @tool
# def exponentiate(base: int, exponent: int) -> int:
#     "Exponentiate the base to the exponent power."
#     return base**exponent


# tools = [multiply, add, exponentiate]

## Agent

In [ ]:
# https://www.youtube.com/watch?v=biS8G8x8DdA[4:51]

# agent 
# agent_executor
# prompt

In [24]:
from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


# Load environment variables from .env file
load_dotenv()

# Load the correct JSON Chat Prompt from the hub
prompt = hub.pull("hwchase17/structured-chat-agent")

# Initialize a ChatOpenAI model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True)

agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,  # Use the conversation memory to maintain context
    handle_parsing_errors=True,  # Handle any parsing errors gracefully
)

initial_message = "You are an AI assistant that can provide helpful answers using available tools.\nIf you are unable to answer, you can use the following tools: Time and Wikipedia."
memory.chat_memory.add_message(SystemMessage(content=initial_message))

while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        break
    memory.chat_memory.add_message(HumanMessage(content=user_input))
    response = agent_executor.invoke({"input": user_input})
    print("Bot:", response["output"])
    memory.chat_memory.add_message(AIMessage(content=response["output"]))

c:\Users\Nilay Kumar\Desktop\langchain_app\venv\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)




> Entering new AgentExecutor chain...


ValueError: Unexpected message with type <class 'langchain_core.messages.system.SystemMessage'> at the position 1.

In [ ]:
# Invoking the model

# Pass in the input
# You pass in the tools
# You pass in the message history
# You pass in the scratchpad



In [ ]:
# Adding Chat History to Messages
# How chain Works
# Promppting

In [ ]:
import os
from dotenv import load_dotenv
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            """
            You are a Helpful AI Assistant which tries 
            their best to help the user with their queries,
            using their question and the context 
            from the conversation.
            """
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

chain = prompt | model | StrOutputParser()


while True:
    query = input("You: ")
    if query.lower() == "exit":
        break
    print("You: ", query)
    response = chain.invoke({"question":query, "chat_history": memory.chat_memory})
    memory.chat_memory.add_message(HumanMessage(content=query))
    print(f"AI: {response}")
    memory.chat_memory.add_message(AIMessage(content=response))

In [ ]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

# Load environment variables from .env
load_dotenv()

# Create a ChatOpenAI model
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

# Define prompt templates (no need for separate Runnable chains)
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a comedian who tells jokes about {topic}."),
        ("human", "Tell me {joke_count} jokes."),
    ]
)

# Create the combined chain using LangChain Expression Language (LCEL)
chain = prompt_template | model | StrOutputParser()
# chain = prompt_template | model

# Run the chain
result = chain.invoke({"topic": "lawyers", "joke_count": 3})

# Output
print(result)

In [ ]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnableSequence

# Load environment variables from .env
load_dotenv()

# Create a ChatOpenAI model
model = ChatOpenAI(model="gpt-4")

# Define prompt templates
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a comedian who tells jokes about {topic}."),
        ("human", "Tell me {joke_count} jokes."),
    ]
)

# Create individual runnables (steps in the chain)
format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x: model.invoke(x.to_messages()))
parse_output = RunnableLambda(lambda x: x.content)

# Create the RunnableSequence (equivalent to the LCEL chain)
chain = RunnableSequence(first=format_prompt, middle=[invoke_model], last=parse_output)

# Run the chain
response = chain.invoke({"topic": "lawyers", "joke_count": 3})

# Output
print(response)

In [ ]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda

# Load environment variables from .env
load_dotenv()

# Create a ChatOpenAI model
model = ChatOpenAI(model="gpt-4o")

# Define prompt templates
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a comedian who tells jokes about {topic}."),
        ("human", "Tell me {joke_count} jokes."),
    ]
)

# Define additional processing steps using RunnableLambda
uppercase_output = RunnableLambda(lambda x: x.upper())
count_words = RunnableLambda(lambda x: f"Word count: {len(x.split())}\n{x}")

# Create the combined chain using LangChain Expression Language (LCEL)
chain = prompt_template | model | StrOutputParser() | uppercase_output | count_words

# Run the chain
result = chain.invoke({"topic": "lawyers", "joke_count": 3})

# Output
print(result)

In [ ]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_openai import ChatOpenAI

# Load environment variables from .env
load_dotenv()

# Create a ChatOpenAI model
model = ChatOpenAI(model="gpt-4o")

# Define prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert product reviewer."),
        ("human", "List the main features of the product {product_name}."),
    ]
)


# Define pros analysis step
def analyze_pros(features):
    pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            (
                "human",
                "Given these features: {features}, list the pros of these features.",
            ),
        ]
    )
    return pros_template.format_prompt(features=features)


# Define cons analysis step
def analyze_cons(features):
    cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            (
                "human",
                "Given these features: {features}, list the cons of these features.",
            ),
        ]
    )
    return cons_template.format_prompt(features=features)


# Combine pros and cons into a final review
def combine_pros_cons(pros, cons):
    return f"Pros:\n{pros}\n\nCons:\n{cons}"


# Simplify branches with LCEL
pros_branch_chain = (
    RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser()
)

cons_branch_chain = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)

# Create the combined chain using LangChain Expression Language (LCEL)
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(branches={"pros": pros_branch_chain, "cons": cons_branch_chain})
    | RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"]))
)

# Run the chain
result = chain.invoke({"product_name": "MacBook Pro"})

# Output
print(result)